In [1]:
# part1
from pyspark import SparkConf,SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import*
from pyspark.sql.functions import*
from pyspark import SparkConf,SparkContext

In [2]:
#sc.stop()
sc = SparkContext("local", "Simple App")
spark = SQLContext(sc)
spark_conf = SparkConf().setMaster('local').setAppName('DEMO40')
spark._jvm.net.snowflake.spark.snowflake.SnowflakeConnectorUtils.enablePushdownSession(spark._jvm.org.apache.spark.sql.SparkSession.builder().getOrCreate())
sc._jvm.net.snowflake.spark.snowflake.SnowflakeConnectorUtils.enablePushdownSession(sc._jvm.org.apache.spark.sql.SparkSession.builder().getOrCreate())


In [3]:
sfOptions={'sfURL':'xxxx.snowflakecomputing.com',
           'sfUser':'xxxx',
           'sfPassword': 'xxxx',
           'sfDatabase':'sales',
           'sfSchema':'public',
           'sfRole':'xxxx',
           'sfWarehouse':'xxxx'}

sfSource='net.snowflake.spark.snowflake'

In [4]:
df_nation=spark.read.format(sfSource) \
    .options(**sfOptions) \
    .option("dbtable","nation") \
    .load()

df_region=spark.read.format(sfSource) \
    .options(**sfOptions) \
    .option("dbtable","region") \
    .load()

df_cust=spark.read.format(sfSource) \
    .options(**sfOptions) \
    .option("dbtable","customer") \
    .load()


In [5]:
df_loc = df_nation.join(df_region, df_nation['N_REGIONKEY'] == df_region['R_REGIONKEY']) 


In [6]:
df_cl = df_loc.join(df_cust, df_loc['N_NATIONKEY'] == df_cust['C_NATIONKEY']) \
         .filter(col('R_NAME') == 'AFRICA') \
         .select('C_MKTSEGMENT') \
         .groupBy('C_MKTSEGMENT').count()


In [7]:
%%time
df_cl.show()

+------------+------+
|C_MKTSEGMENT| count|
+------------+------+
|   MACHINERY|601756|
|  AUTOMOBILE|602286|
|   FURNITURE|600983|
|   HOUSEHOLD|601355|
|    BUILDING|601798|
+------------+------+

CPU times: user 0 ns, sys: 10 ms, total: 10 ms
Wall time: 10.5 s


In [8]:
df_cl.printSchema()

root
 |-- C_MKTSEGMENT: string (nullable = true)
 |-- count: long (nullable = false)



In [9]:

df_cl_single = df_loc.join(df_cust, df_loc['N_NATIONKEY'] == df_cust['C_NATIONKEY']) \
         .filter(df_cust['C_CUSTKEY'] == '123456') \
         .select('C_MKTSEGMENT') \
         .groupBy('C_MKTSEGMENT').count()


In [10]:
%%time
df_cl_single.show()

+------------+-----+
|C_MKTSEGMENT|count|
+------------+-----+
|  AUTOMOBILE|    1|
+------------+-----+

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.96 s
